## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather-database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,46.99,42,13,4.00,few clouds
1,1,Luderitz,NaN,-26.6481,15.1594,80.20,52,6,29.12,clear sky
2,2,Hobart,AU,-42.8794,147.3294,55.99,76,0,4.61,clear sky
3,3,Port Alfred,ZA,-33.5906,26.8910,66.88,78,100,11.77,overcast clouds
4,4,Tuatapere,NZ,-46.1333,167.6833,53.01,63,6,7.00,clear sky


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?50
What is the maximum temperature you would like for your trip?91


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Luderitz,NaN,-26.6481,15.1594,80.20,52,6,29.12,clear sky
2,2,Hobart,AU,-42.8794,147.3294,55.99,76,0,4.61,clear sky
3,3,Port Alfred,ZA,-33.5906,26.8910,66.88,78,100,11.77,overcast clouds
4,4,Tuatapere,NZ,-46.1333,167.6833,53.01,63,6,7.00,clear sky
5,5,Bluff,NZ,-46.6000,168.3333,53.01,63,69,7.00,broken clouds
7,7,Vaini,TO,-21.2000,-175.2000,62.60,100,20,1.14,few clouds
10,10,Avarua,CK,-21.2078,-159.7750,78.80,100,90,11.50,moderate rain
11,11,Lagoa,PT,39.0500,-27.9833,59.74,85,77,16.28,broken clouds
12,12,Yulara,AU,-25.2406,130.9889,75.20,31,0,4.61,clear sky
13,13,Cape Town,ZA,-33.9258,18.4232,68.00,68,20,27.63,few clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                453
City                   453
Country                447
Lat                    453
Lng                    453
Max Temp               453
Humidity               453
Cloudiness             453
Wind Speed             453
Current Description    453
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(inplace=True)
clean_df.count()

City_ID                447
City                   447
Country                447
Lat                    447
Lng                    447
Max Temp               447
Humidity               447
Cloudiness             447
Wind Speed             447
Current Description    447
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hobart,AU,55.99,clear sky,-42.8794,147.3294,
3,Port Alfred,ZA,66.88,overcast clouds,-33.5906,26.8910,
4,Tuatapere,NZ,53.01,clear sky,-46.1333,167.6833,
5,Bluff,NZ,53.01,broken clouds,-46.6000,168.3333,
7,Vaini,TO,62.60,few clouds,-21.2000,-175.2000,
10,Avarua,CK,78.80,moderate rain,-21.2078,-159.7750,
11,Lagoa,PT,59.74,broken clouds,39.0500,-27.9833,
12,Yulara,AU,75.20,clear sky,-25.2406,130.9889,
13,Cape Town,ZA,68.00,few clouds,-33.9258,18.4232,
14,Seydi,TM,63.75,clear sky,39.4816,62.9137,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index , row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{lng}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url , params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found ... skipping.')
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df['Hotel Name'].dropna(inplace=True)
clean_hotel_df.count()

City                   447
Country                447
Max Temp               447
Current Description    447
Lat                    447
Lng                    447
Hotel Name             447
dtype: int64

In [28]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))